In [ ]:
import time
from rdflib import Graph, URIRef, Literal, Namespace, XSD
from rdflib.namespace import RDF, RDFS, DCTERMS, SKOS
import requests
import spotlight
from multiprocessing.dummy import Pool as ThreadPool  # Usamos hilos en lugar de procesos

# Namespaces
schema = Namespace("https://schema.org/")
dbpedia = Namespace("http://dbpedia.org/resource/")
dbo = Namespace("http://dbpedia.org/ontology/")
myv = Namespace("http://example.org/myv/")

# Configuración del endpoint de DBpedia Spotlight
DBPEDIA_ENDPOINT = 'https://api.dbpedia-spotlight.org/en/'

# Caché de respuestas para DBpedia
dbpedia_cache = {}

def get_annotations(text, confidence=0.4, support=20):
    """ Obtiene anotaciones de un texto usando DBpedia Spotlight. """
    start_time = time.time()
    try:
        print(f"Anotando texto: {text[:100]}...")
        annotations = spotlight.annotate(DBPEDIA_ENDPOINT + 'annotate', text, confidence=confidence, support=support)
        elapsed_time = time.time() - start_time
        print(f"Se encontraron {len(annotations)} anotaciones en {elapsed_time:.2f} segundos.")
        return [
            {
                'uri': a['URI'],
                'surfaceForm': a['surfaceForm'],
                'similarityScore': a['similarityScore']
            }
            for a in annotations
        ]
    except Exception as e:
        print(f"Error en DBpedia Spotlight: {e}")
        return []

def get_dbpedia_info(uri):
    """ Consulta DBpedia para obtener información adicional de una URI. """
    if uri in dbpedia_cache:
        return dbpedia_cache[uri]  # Usamos caché para evitar consultas repetidas

    print(f"Consultando DBpedia para la URI: {uri}")
    query = f"""
    SELECT ?prop ?value WHERE {{
        VALUES ?uri {{ <{uri}> }}
        VALUES ?prop {{ rdfs:label rdfs:comment dcterms:subject }}
        ?uri ?prop ?value .
        FILTER(LANG(?value) = "es" || LANG(?value) = "en")
    }}
    """
    sparql_endpoint = "http://dbpedia.org/sparql"
    params = {"query": query, "format": "json"}

    start_time = time.time()
    try:
        response = requests.get(sparql_endpoint, params=params, timeout=5)  # Tiempo máximo de espera
        elapsed_time = time.time() - start_time

        if response.status_code == 200:
            data = response.json()
            results = [(result["prop"]["value"], result["value"]["value"]) for result in data["results"]["bindings"]]
            dbpedia_cache[uri] = results  # Guardamos en caché
            print(f"Información encontrada para {uri} en {elapsed_time:.2f} segundos.")
            return results
        else:
            print(f"Error SPARQL ({response.status_code}): {response.text}")
    except Exception as e:
        print(f"Error consultando DBpedia: {e}")
    
    return []

def process_article(article, description):
    """ Procesa un artículo y su descripción para enriquecer el grafo. """
    start_time = time.time()
    enriched_graph = Graph()
    print(f"\nProcesando artículo: {article}")
    print(f"Descripción: {description[:3000]}...")

    # Anotaciones con DBpedia Spotlight
    annotations = get_annotations(str(description))
    for annotation in annotations[:5]:  # Reducimos a 5 anotaciones por artículo
        dbpedia_uri = URIRef(annotation['uri'])
        surface_form = annotation['surfaceForm']
        similarity_score = annotation['similarityScore']

        enriched_graph.add((article, schema.mentions, dbpedia_uri))
        enriched_graph.add((dbpedia_uri, RDFS.label, Literal(surface_form)))
        enriched_graph.add((dbpedia_uri, myv.similarityScore, Literal(similarity_score, datatype=XSD.float)))

        # Obtener información adicional de DBpedia
        info = get_dbpedia_info(dbpedia_uri)
        for prop, value in info:
            enriched_graph.add((dbpedia_uri, URIRef(prop), Literal(value)))

    elapsed_time = time.time() - start_time
    print(f"Procesamiento de {article} completado en {elapsed_time:.2f} segundos.")
    return enriched_graph

# Cargar el grafo RDF original
input_file = "articles_data_cleaned_final_1.ttl"
print(f"\nCargando grafo RDF desde {input_file}...")
g = Graph()
try:
    g.parse(input_file, format="turtle")
    print(f"Grafo cargado correctamente. Número de triples: {len(g)}")
except Exception as e:
    print(f"Error al cargar el grafo RDF: {e}")
    exit()

# Preparar el grafo enriquecido
enriched_graph = Graph()
enriched_graph.bind("dbo", dbo)
enriched_graph.bind("dbpedia", dbpedia)
enriched_graph.bind("myv", myv)
enriched_graph.bind("skos", SKOS)
enriched_graph.bind("schema", schema)

# Extraer artículos y sus descripciones (limitado a 10 artículos para pruebas)
articles = list(g.subjects(RDF.type, schema.Article))[:3000]  
print(f"\nSe procesarán {len(articles)} artículos.")

# Procesar artículos en paralelo usando hilos
print("\nIniciando procesamiento de artículos...")
try:
    with ThreadPool(processes=6) as pool:  # 4 hilos en lugar de procesos
        results = pool.starmap(process_article, [(article, g.value(article, schema.description)) for article in articles if g.value(article, schema.description)])
    print("\nProcesamiento de artículos completado.")
except Exception as e:
    print(f"Error en procesamiento paralelo: {e}")
    exit()

# Combinar los resultados en el grafo enriquecido
print("\nCombinando resultados en el grafo enriquecido...")
for result in results:
    enriched_graph += result
print(f"Grafo enriquecido listo. Número de triples: {len(enriched_graph)}")

# Guardar el grafo enriquecido
output_file = "enriched_articles_data_with_annotations_2.ttl"
print(f"\nGuardando grafo enriquecido en {output_file}...")
try:
    enriched_graph.serialize(output_file, format="turtle")
    print(f"Grafo enriquecido guardado correctamente.")
except Exception as e:
    print(f"Error al guardar el grafo: {e}")

#### CORRECCIÓN DE URI

In [1]:
import re

# Ruta del archivo original y del archivo corregido
input_file = "pre_enriched_annotations.ttl"
output_file = "enriched_annotations.ttl"

# Expresión regular para encontrar y reemplazar en URIs específicas
pattern = re.compile(r"<http://example.org/article/(.*?)>")

with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    for line in infile:
        modified_line = pattern.sub(r"<http://example.org/sa/\1>", line)
        outfile.write(modified_line)

print("Archivo corregido guardado como", output_file)


Archivo corregido guardado como enriched_annotations.ttl


In [2]:
!jupyter nbconvert --to html enrequecimiento_annotation.ipynb

[NbConvertApp] Converting notebook enrequecimiento_annotation.ipynb to html
[NbConvertApp] Writing 299528 bytes to enrequecimiento_annotation.html
